In [16]:
import pandas as pd
import glob
from functools import reduce

audl = pd.concat([pd.read_csv(dfteam, index_col=0) for dfteam in glob.glob('../../data/*/*.csv')])

In [17]:
audl[(audl.Action.isin(['Pull','PullOb']))&(audl.Defender=='Jimmy Robin')]

,Date/Time,Opponent,Point Elapsed Seconds,Line,Our Score - End of Point,Their Score - End of Point,Event Type,Action,Passer,Receiver,...,Begin Y,End Area,End X,End Y,Distance Unit of Measure,Absolute Distance,Lateral Distance,Toward Our Goal Distance,Teamname,Tournament
1081,2017-05-06 18:04,Madison Radicals,48,D,10,13,Defense,Pull,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chicago Wildfire,AUDL 2017
1333,2017-05-13 19:03,Minnesota Wind Chill,41,D,5,6,Defense,PullOb,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chicago Wildfire,AUDL 2017
1345,2017-05-13 19:03,Minnesota Wind Chill,19,D,6,7,Defense,Pull,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chicago Wildfire,AUDL 2017


In [37]:
audl['Begin Area'].unique()
audl['Action'].unique()

array(['Catch', 'Throwaway', 'Goal', 'Pull', 'D', 'Drop',
       'EndOfFirstQuarter', 'Halftime', 'EndOfThirdQuarter', 'GameOver',
       'PullOb', 'Stall', 'Callahan', 'MiscPenalty', 'EndOfFourthQuarter',
       'EndOfOvertime', 'OpponentPull', 'OpponentCatch', 'OpponentPullOb'],
      dtype=object)

In [57]:
audl[audl.Action=='MiscPenalty'][['Passer','Receiver','Defender']].head()

,Passer,Receiver,Defender
1770,Adam Drews,NaN,NaN
3450,Adam Drews,NaN,NaN
3551,Davis Whitehead,NaN,NaN
4634,Ben Nelson,NaN,NaN
4739,NaN,NaN,NaN


In [87]:
audl[audl.Action.isin(['Pull','PullOb'])][['Passer','Defender','Receiver','End X']].head()
audl[audl.Action.isin(['Pull','PullOb'])&~pd.isnull(audl['End Y'])][['Begin Area',
       'Begin X', 'Begin Y', 'End Area', 'End X', 'End Y',
       'Distance Unit of Measure', 'Absolute Distance', 'Lateral Distance',
       'Toward Our Goal Distance']].head()

,Begin Area,Begin X,Begin Y,End Area,End X,End Y,Distance Unit of Measure,Absolute Distance,Lateral Distance,Toward Our Goal Distance
2044,Endzone B,0.203,0.586,Field,0.053,0.138,yds,83.3,23.9,79.9
2063,Endzone A,0.717,0.593,Endzone B,0.466,0.621,yds,95.0,1.5,95.0
2076,Endzone B,0.331,0.143,Field,0.111,0.482,yds,79.8,18.1,77.7
2088,Endzone A,0.803,0.706,Endzone B,0.293,0.558,yds,90.1,7.9,89.8
2126,Endzone A,0.752,0.711,Endzone B,0.386,0.338,yds,94.8,19.9,92.7


In [13]:
audl.columns

Index(['Date/Time', 'Opponent', 'Point Elapsed Seconds', 'Line',
       'Our Score - End of Point', 'Their Score - End of Point', 'Event Type',
       'Action', 'Passer', 'Receiver', 'Defender', 'Hang Time (secs)',
       'Player 0', 'Player 1', 'Player 2', 'Player 3', 'Player 4', 'Player 5',
       'Player 6', 'Player 7', 'Player 8', 'Player 9', 'Player 10',
       'Player 11', 'Player 12', 'Player 13', 'Player 14', 'Player 15',
       'Player 16', 'Player 17', 'Player 18', 'Player 19', 'Player 20',
       'Player 21', 'Player 22', 'Player 23', 'Player 24', 'Player 25',
       'Player 26', 'Player 27', 'Elapsed Time (secs)', 'Begin Area',
       'Begin X', 'Begin Y', 'End Area', 'End X', 'End Y',
       'Distance Unit of Measure', 'Absolute Distance', 'Lateral Distance',
       'Toward Our Goal Distance', 'Teamname', 'Tournament'],
      dtype='object')

In [9]:
def GetStat(df_slice, player_field,stat_name,aggfunc):
    stat = audl[df_slice].groupby(['Tournament',player_field,'Teamname']).apply(aggfunc)
    stat = stat.reset_index().rename(columns={'Tournament':'Year',player_field:'Name',0:stat_name})
    return stat

def GetCountStat(df_slice, player_field,stat_name):
    return GetStat(df_slice, player_field,stat_name,lambda x : len(x))

count_stat_list= [
                    # Catching
                    [(audl.Action=='Goal'), 'Receiver','Goals'],
                    [(audl.Action.isin(['Goal','Catch'])), 'Receiver','Catches'],
                    [(audl.Action=='Drop'), 'Receiver','Drops'],
                    # Throwing
                    [(audl.Action=='Goal'), 'Passer','Assists'],
                    [([True]*len(audl)), 'Passer','Throws'],
                    [(audl.Action.isin(['Goal','Catch'])), 'Passer','Completions'],
                    [(audl.Action.isin(['Throwaway','Callahan'])),'Passer','Throwaways'],
                    [(audl.Action=='Drop'), 'Passer','Throws Dropped'],
                    [(audl.Action.isin(['Goal','Catch'])&~pd.isnull(audl['Absolute Distance'])), 'Passer','Throws Recorded with Yardage'],
                    [(audl.Action=='MiscPenalty'),'Passer','Fouls Drawn'],
                    [(audl.Action=='Stall'),'Passer','Stalls'],

                    # Defense
                    [(audl.Action.isin(['D','Callahan'])), 'Defender', 'Blocks'],
                    [(audl.Action=='Callahan'), 'Defender','Callahans'],
                    [audl.Action.isin(['Pull','PullOb']), 'Defender', 'Pulls'],
                    [(audl.Action=='PullOb'), 'Defender', 'Pulls (Out-of-bounds)'],
                    [audl.Action.isin(['Pull','PullOb'])&~pd.isnull(audl['Absolute Distance']), 'Defender','Pulls Recorded with Yardage'],
                    [(audl.Action=='Pull')&~pd.isnull(audl['Absolute Distance']), 'Defender','Pulls (Inbounds) Recorded with Yardage'],

]


other_stat_list = [
                    [audl.Action.isin(['Goal','Catch'])&~pd.isnull(audl['Absolute Distance']),
                     'Passer',
                     'Throwing Yards',
                     lambda x : sum(x['Absolute Distance'])],
                    [audl.Action.isin(['Goal','Catch'])&~pd.isnull(audl['Lateral Distance']),
                     'Passer',
                     'Lateral Throwing Yards',
                     lambda x : sum(x['Lateral Distance'])],
                    [audl.Action.isin(['Goal','Catch'])&~pd.isnull(audl['Toward Our Goal Distance']),
                     'Passer',
                     'Forward Throwing Yards',
                     lambda x : sum(x['Toward Our Goal Distance'])],

                    [audl.Action.isin(['Pull','PullOb'])&~pd.isnull(audl['Absolute Distance']),
                     'Defender',
                     'Pull Yards',
                     lambda x : sum(x['Absolute Distance'])],

                    [(audl.Action=='Pull')&~pd.isnull(audl['Absolute Distance']),
                     'Defender',
                     'Pull Yards (Inbounds)',
                     lambda x : sum(x['Absolute Distance'])]
    
                   ]

ind_stats = [GetCountStat(i,j,k) for i,j,k in count_stat_list] + \
            [GetStat(i,j,k,l) for i,j,k,l in other_stat_list]
    

base_stats = reduce(lambda  left,right: pd.merge(left,right,on=['Year','Teamname','Name'],
                                                 how='outer'), 
                                                 ind_stats).fillna(0)

In [10]:
def SafeDivide(x,y):
    if y!=0:
        return x/y
    else:
        return 0
    
secondary_stats = [ 
                    ['Completion Percentage', lambda x : 100*SafeDivide(x.Completions,x.Throws)],
                    ['Catches Per Goals', lambda x : SafeDivide(x.Catches,x.Goals)],
                    ['Throws Per Assist', lambda x : SafeDivide(x.Throws,x.Assists)],
                    ['Throwaway Percentage', lambda x : SafeDivide(x.Throwaways,x.Throws)],
                    ['Drop Percentage', lambda x : SafeDivide(x.Drops,(x.Drops + x.Catches))],  
                    ['Yards Per Throw', lambda x : SafeDivide(x['Throwing Yards'],x['Throws Recorded with Yardage'])],    
                    ['Yards Per Pull', lambda x : SafeDivide(x['Pull Yards'],x['Pulls Recorded with Yardage'])],  
                    ['Yards Per Pull (Inbounds)', lambda x : SafeDivide(x['Pull Yards (Inbounds)'],x['Pulls (Inbounds) Recorded with Yardage'])],    

                 ]

for sec_stat,aggfunc in secondary_stats:
    base_stats[sec_stat] = base_stats.apply(aggfunc,axis=1)

In [124]:
def CheckTopN(col_name,n=50):
    return base_stats.sort_values(col_name,ascending=False).iloc[:n][['Name','Year','Teamname',col_name]]

CheckTopN('Throws Recorded with Yardage')
# CheckTopN('Fouls Against')
# CheckTopN('Throws Dropped')
# CheckTopN('Pull Yards')
# CheckTopN('Yards Per Throw')

# base_stats.groupby('Name').apply(sum).sort_values('Callahans',ascending=False)

,Name,Year,Teamname,Yards Per Throw
3231,Zakarie Massi,AUDL 2018,Montreal Royal,61.050000
2029,Dave Wiseman,AUDL 2017,Madison Radicals,25.684615
3169,Ted Barnett,AUDL 2018,Los Angeles Aviators,23.500000
1991,Christian Foster,AUDL 2017,Montreal Royal,22.819231
2441,Ryan Tucker,AUDL 2017,Madison Radicals,22.414286
796,Kevin Smith,AUDL 2015,San Jose Spiders,22.368478
2297,Marquis Mason,AUDL 2017,Madison Radicals,22.200000
2286,Mark Elbogen,AUDL 2017,Los Angeles Aviators,22.195082
2243,Kevin Groulx,AUDL 2017,Montreal Royal,22.150000
2608,Antoine Genest,AUDL 2018,Montreal Royal,22.044681


In [13]:
audl[(audl.Action.isin(['Pull','PullOb']))&(audl.Defender=='Jimmy Robin')]

,Absolute Distance,Action,Assists,Begin Area,Begin X,Begin Y,Blocks,Callahans,Catches,Catches Per Goals,...,Throws,Throws Dropped,Throws Per Assist,Throws Recorded with Yardage,Tournament,Toward Our Goal Distance,Yards Per Pull,Yards Per Pull (Inbounds),Yards Per Throw,Year
1081,NaN,Pull,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,AUDL 2017,NaN,NaN,NaN,NaN,NaN
1333,NaN,PullOb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,AUDL 2017,NaN,NaN,NaN,NaN,NaN
1345,NaN,Pull,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,AUDL 2017,NaN,NaN,NaN,NaN,NaN


In [7]:
base_stats[base_stats.Name=='Jimmy Robin'].to_excel('Jimmy.xlsx')

In [111]:
# base_stats[base_stats.Throws>52].to_csv('../../data/processed//IndividualStats_52throws.csv')